##YOLOv5를 다운로드

In [1]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 15874, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 15874 (delta 47), reused 45 (delta 15), pack-reused 15768
Receiving objects: 100% (15874/15874), 14.69 MiB | 35.22 MiB/s, done.
Resolving deltas: 100% (10856/10856), done.


##YOLOv5을 위한 패키지를 설치

In [2]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.9/606.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="WqJ0pnj2vv5G2WtOPiQV")
project = rf.workspace("joseph-nelson").project("pistols")
dataset = project.version(1).download("yolov5")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.6 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=5b207d6952d50321b5ae0bd99f5491c0a160e3a7a73f5b40d327378a89852ba5
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.0
    Uninstalling pyparsing-3.1.0:
      Su

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Pistols-1 in yolov5pytorch:: 100%|██████████| 5945/5945 [00:00<00:00, 9544.55it/s] 


## data.yaml 파일 안의 train 경로와 valid 경로가 각각 train: ../train/images, valid: ../valid/images로 설정되어 있는 것을 확인할 수 있다. 경로가 정확하지 않기 때문에 경로를 바꿔준다.

In [4]:
%cat /content/dataset/data.yaml

cat: /content/dataset/data.yaml: No such file or directory


In [5]:
%cd /
from glob import glob
img_list = glob('/content/yolov5/Pistols-1/export/images/*.jpg')
print(len(img_list))

/
2971


In [6]:
from sklearn.model_selection import train_test_split
train_img_list, val_img_list = train_test_split(img_list, test_size = 0.2, random_state = 2000)
print(len(train_img_list), len(val_img_list))

2376 595


##나누어 주었던 train set과 valid set을 각각 txt파일로 만들어준다. 그러면 이미지의 주소들이 txt파일에 적히게 된다.

In [7]:
with open('/content/yolov5/Pistols-1/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/yolov5/Pistols-1/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [8]:
import yaml

with open('/content/yolov5/Pistols-1/data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/content/yolov5/Pistols-1/train.txt'
data['val'] = '/content/yolov5/Pistols-1/val.txt'

with open('/content/yolov5/Pistols-1/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'names': ['pistol'], 'nc': 1, 'train': 'Pistols-1/train/images', 'val': 'Pistols-1/valid/images'}
{'names': ['pistol'], 'nc': 1, 'train': '/content/yolov5/Pistols-1/train.txt', 'val': '/content/yolov5/Pistols-1/val.txt'}


In [9]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /content/yolov5/Pistols-1/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name results

/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/yolov5/Pistols-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-203-g0897415 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1

In [17]:
!python detect.py --weights /content/yolov5/runs/train/results/weights/best.pt --img 416 --conf 0.5 --source ../gun.mp4

detect: weights=['/content/yolov5/runs/train/results/weights/best.pt'], source=../gun.mp4, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-203-g0897415 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/586) /content/gun.mp4: 416x224 1 pistol, 85.7ms
video 1/1 (2/586) /content/gun.mp4: 416x224 1 pistol, 6.4ms
video 1/1 (3/586) /content/gun.mp4: 416x224 1 pistol, 6.4ms
video 1/1 (4/586) /content/gun.mp4: 416x224 1 pistol, 6.5ms
video 1/1 (5/586) /content/gun.mp4: 416x224 1 pistol, 6.7ms
video 1/1 (6/586) /c